In [1]:
import numpy as np
import pandas as pd
from senticnet5 import senticnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re
from scipy import sparse
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_excel('hand8_k_random.xlsx')
print(len(df))

8501


In [3]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multioutput import ClassifierChain

In [4]:
def convtodec(x):
    val = 128
    ret = 0
    for y in x:
        if(y):
            ret+=val
        val=val>>1
    return ret
labelpowerset = []
for row in df.iterrows():
    tmp = row[1][1:9].tolist()
    labelpowerset.append(convtodec(tmp))
print(len(labelpowerset))

8501


In [5]:
cntpowerset = {}
for val in labelpowerset:
    if(val in cntpowerset):
        cntpowerset[val]+=1
    else:
        cntpowerset[val]=1

cntpowersetlist = []
for key,val in cntpowerset.items():
    cntpowersetlist.append((val,key))
cntpowersetlist.sort(reverse=True)
print(len(cntpowersetlist))

95


In [6]:
label = {}
revlabel = {}
cnt = 0
for val in cntpowersetlist:
    label[val[1]] = cnt
    revlabel[cnt] = val[1]
    cnt+=1
powset = [label[x] for x in labelpowerset]
df['powerset'] = powset

In [7]:
df['powerset'].value_counts()

0     1870
1      998
2      772
3      711
4      348
      ... 
85       1
93       1
86       1
94       1
91       1
Name: powerset, Length: 95, dtype: int64

In [8]:
df_tmp = df
print(len(df_tmp))

8501


In [9]:
def get_senticnet(word,em):
    em = '#'+em.lower()
    if(senticnet[word][4]==em or senticnet[word][5]==em):
        return 1
    else:
        return 0

In [10]:
X = df_tmp['Text']
Y = df_tmp['powerset']

In [11]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df_tmp['Text'])
print(len(vectorizer.vocabulary_))

17402


In [12]:
kf = KFold(n_splits = 10)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [13]:
def evaluation(score_list,predict_score_list):
    filter_corr = []
    exmatch = 0
    atleast1 = 0
    md1 = 0
    one_f = 0
    more_f = 0
    zero_f = 0
    sm = 0
    sdensity = 0
    hammval = 0
    test_len = len(predict_score_list[0])
    for j in range(test_len):
        cnt=0
        for i in range(8):
            hammval+=(score_list[i][j] ^ int(predict_score_list[i][j]))
            if(score_list[i][j]==1):
                cnt+=1
                sm+=1
        sdensity+=cnt/8
        if(cnt==0):
            zero_f+=1
        if(cnt==1):
            one_f+=1
        if(cnt>1):
            more_f+=1
        for i in range(8):
            mf = True
            if(int(predict_score_list[i][j])!=score_list[i][j]):
                mf=False
                break
        if(mf==True):
            exmatch+=1
            filter_corr.append(j)
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                atleast1+=1
                break
        mf = False
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                if(mf==True):
                    md1+=1
                    filter_corr.append(j)
                    break
                mf=True
    #print("Label Cardinality: "+ str(sm/test_len))
    #print("Label Density: "+ str(sdensity/test_len))
    print("Hamming Loss: "+str(hammval/(test_len*8)))
    hamlos = hammval/(test_len*8)
    print("Exact Prediction: "+str(exmatch/test_len))
    sub_accu = exmatch/test_len
    #print("At least one label predicted: "+str(atleast1/(test_len-zero_f)))
    #print("More than one label predicted: "+str(md1/more_f))
    tp_sum = 0
    fp_sum = 0
    fn_sum = 0
    macro_preci = 0
    macro_recall = 0
    macro_f1 = 0
    for i in range(len(score_list)):
        tmp = confusion_matrix(score_list[i],predict_score_list[i])
        tp_sum+=tmp[0][0]
        fp_sum+=tmp[0][1]
        fn_sum+=tmp[1][0]
        macro_preci_tmp=tmp[0][0]/(tmp[0][0]+tmp[0][1])
        macro_recall_tmp=tmp[0][0]/(tmp[0][0]+tmp[1][0])
        macro_f1 += ((2*macro_preci_tmp*macro_recall_tmp)/(macro_preci_tmp+macro_recall_tmp))
        macro_preci+=macro_preci_tmp
        macro_recall+=macro_recall_tmp
        #print(macro_f1)
    micro_preci = tp_sum/(tp_sum+fp_sum)
    micro_recall = tp_sum/(tp_sum+fn_sum)
    micro_f1 = (2*micro_preci*micro_recall)/(micro_preci+micro_recall)
    macro_preci/=8
    macro_recall/=8
    macro_f1/=8
    #print(micro_preci,micro_recall,micro_f1)
    #print(macro_preci,macro_recall,macro_f1)
    print("Macro F-Score: "+str(macro_f1))
    print("Micro F-Score: "+str(micro_f1))
    col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
    tmp = 0
    for i in range(len(score_list)):
        score = accuracy_score(score_list[i],predict_score_list[i]) 
        #print(col_names[i]+" accuracy: "+str(score))
        tmp += score
    print("Average Accuracy: " + str(tmp/8))
    avg_accu = tmp/8
    return (hamlos,sub_accu,macro_f1,micro_f1,avg_accu)

In [14]:
def convert2bin(xx):
    ret = []
    for i in range(8):
        if(xx & (1<<i)):
            ret.append(1)
        else:
            ret.append(0)
    #print(xx)
    ret.reverse()
    #print(ret)
    return ret

In [15]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    clf = RandomForestClassifier(n_estimators=300)
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = np.array(Y.iloc[train_index].tolist()),np.array(Y.iloc[test_index].tolist())
    print("k_fold validation: " + str(cnt))
    cnt+=1
    
    x_train = vectorizer.transform(x_train)
    x_test = vectorizer.transform(x_test)
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    score_list = []
    predict_score_list = []
    for i in range(len(y_test)):
        score_list.append(convert2bin(revlabel[y_test[i]]))
        predict_score_list.append(convert2bin(revlabel[y_pred[i]]))
    np_score_list = np.array(score_list)
    transpose = np_score_list.T
    score_list = transpose.tolist()

    np_predict_score_list = np.array(predict_score_list)
    transpose = np_predict_score_list.T
    predict_score_list = transpose.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(7650, 17402) (851, 17402)
(7650,) (851,)
Hamming Loss: 0.16480611045828436
Exact Prediction: 0.2890716803760282
Macro F-Score: 0.8694673928987255
Micro F-Score: 0.902281832433374
Average Accuracy: 0.8351938895417157


k_fold validation: 2
(7651, 17402) (850, 17402)
(7651,) (850,)
Hamming Loss: 0.175
Exact Prediction: 0.28
Macro F-Score: 0.8649434099652007
Micro F-Score: 0.8961967899511514
Average Accuracy: 0.8250000000000001


k_fold validation: 3
(7651, 17402) (850, 17402)
(7651,) (850,)
Hamming Loss: 0.18
Exact Prediction: 0.2447058823529412
Macro F-Score: 0.8597889457950241
Micro F-Score: 0.8926315789473684
Average Accuracy: 0.8199999999999998


k_fold validation: 4
(7651, 17402) (850, 17402)
(7651,) (850,)
Hamming Loss: 0.17161764705882354
Exact Prediction: 0.2858823529411765
Macro F-Score: 0.8654152234012874
Micro F-Score: 0.8977123323691822
Average Accuracy: 0.8283823529411763


k_fold validation: 5
(7651, 17402) (850, 17402)
(7651,) (850,)
Hamming Loss: 0.1

In [16]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [17]:
df_rfc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_rfc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.164806,0.289072,0.869467,0.902282,0.835194
1,2,0.175000,0.280000,0.864943,0.896197,0.825000
2,3,0.180000,0.244706,0.859789,0.892632,0.820000
3,4,0.171618,0.285882,0.865415,0.897712,0.828382
4,5,0.173529,0.268235,0.862801,0.897015,0.826471
5,6,0.172059,0.261176,0.866411,0.897656,0.827941
6,7,0.171765,0.250588,0.863762,0.897580,0.828235
7,8,0.172794,0.289412,0.865935,0.897281,0.827206
8,9,0.171176,0.280000,0.865576,0.897877,0.828824
9,10,0.167647,0.280000,0.872885,0.900332,0.832353


In [18]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    clf = SVC()
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = np.array(Y.iloc[train_index].tolist()),np.array(Y.iloc[test_index].tolist())
    print("k_fold validation: " + str(cnt))
    cnt+=1
    
    x_train = vectorizer.transform(x_train)
    x_test = vectorizer.transform(x_test)
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    score_list = []
    predict_score_list = []
    for i in range(len(y_test)):
        score_list.append(convert2bin(revlabel[y_test[i]]))
        predict_score_list.append(convert2bin(revlabel[y_pred[i]]))
    np_score_list = np.array(score_list)
    transpose = np_score_list.T
    score_list = transpose.tolist()

    np_predict_score_list = np.array(predict_score_list)
    transpose = np_predict_score_list.T
    predict_score_list = transpose.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(7650, 17402) (851, 17402)
(7650,) (851,)
Hamming Loss: 0.172737955346651
Exact Prediction: 0.26556991774383076
Macro F-Score: 0.8438113491819751
Micro F-Score: 0.8976323119777159
Average Accuracy: 0.8272620446533491


k_fold validation: 2
(7651, 17402) (850, 17402)
(7651,) (850,)
Hamming Loss: 0.18573529411764705
Exact Prediction: 0.2611764705882353
Macro F-Score: 0.8387801257659812
Micro F-Score: 0.8897809581987958
Average Accuracy: 0.8142647058823529


k_fold validation: 3
(7651, 17402) (850, 17402)
(7651,) (850,)
Hamming Loss: 0.1814705882352941
Exact Prediction: 0.23058823529411765
Macro F-Score: 0.8428557632258149
Micro F-Score: 0.8918871561240582
Average Accuracy: 0.8185294117647058


k_fold validation: 4
(7651, 17402) (850, 17402)
(7651,) (850,)
Hamming Loss: 0.17911764705882352
Exact Prediction: 0.2529411764705882
Macro F-Score: 0.8427652931762232
Micro F-Score: 0.8934942287513117
Average Accuracy: 0.8208823529411765


k_fold validation: 5
(7651, 17402) (8

In [19]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [20]:
df_svc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_svc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.172738,0.265570,0.843811,0.897632,0.827262
1,2,0.185735,0.261176,0.838780,0.889781,0.814265
2,3,0.181471,0.230588,0.842856,0.891887,0.818529
3,4,0.179118,0.252941,0.842765,0.893494,0.820882
4,5,0.181912,0.248235,0.837810,0.892144,0.818088
5,6,0.178235,0.249412,0.841671,0.894222,0.821765
6,7,0.177794,0.236471,0.840174,0.894179,0.822206
7,8,0.178235,0.276471,0.846452,0.894315,0.821765
8,9,0.181618,0.256471,0.839340,0.891847,0.818382
9,10,0.183971,0.256471,0.835327,0.890656,0.816029
